In [ ]:
# Load YOLO, train it 
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import numpy as np

model = YOLO('server/12l-train-best.pt')

In [ ]:
%time
img = cv2.imread('model/6.png')
print(img.shape)

# Инференс с использованием модели
results = model(img, imgsz=512, iou=0.1, conf=0.3, verbose=True)
print(results[0].boxes.conf)
is_no_mask = True if results[0].masks is None else False
print(is_no_mask)

In [ ]:
# Получение классов и имен классов
classes = results[0].boxes.cls.cpu().numpy()
class_names = results[0].names

confs = results[0].boxes.conf.cpu().numpy()

# Получение бинарных масок и их количество
masks = results[0].masks.data  # Формат: [число масок, высота, ширина]
num_masks = masks.shape[0]

# Коралловый цвет для каждой маски
colors = [(90, 80, 252) for _ in range(num_masks)] # BGR
font_color = (255, 0, 0) # BGR

# Создание изображения для отображения масок
mask_overlay = np.zeros_like(img)

labeled_image = img.copy()


# Добавление подписей и контура к маскам
for i in range(num_masks):
    color = colors[i] # Извлекаем цвет
    mask = masks[i].cpu()
    
    # Изменение размера маски до размеров исходного изображения с использованием метода ближайших соседей
    mask_resized = cv2.resize(np.array(mask), (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)
    
    # Получение класса для текущей маски
    class_index = int(classes[i])
    class_name = class_names[class_index]
    
    # Добавление подписи к маске
    mask_contours, _ = cv2.findContours(mask_resized.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Накладываем видимость контуров
    cv2.drawContours(labeled_image, mask_contours, -1, color, 1)
    
    x_coord = mask_contours[0][:, 0, 0].mean() # Минимум по X (лево-право)
    y_coord = mask_contours[0][:, 0, 1].mean() # Середина по Y (вверх-вниз)
    cv2.putText(labeled_image, f"{confs[i]:.2f}", (int(x_coord), int(y_coord)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, font_color, 1)

# Отобразите итоговое изображение с наложенными масками и подписями
plt.figure(figsize=(8, 8), dpi=150)
labeled_image = cv2.cvtColor(labeled_image, cv2.COLOR_BGR2RGB)
plt.imshow(labeled_image)
plt.axis('off')
plt.show()

In [ ]:
# Получение классов и имен классов
classes = results[0].boxes.cls.cpu().numpy()
class_names = results[0].names

confs = results[0].boxes.conf.cpu().numpy()

# Получение бинарных масок и их количество
masks = results[0].masks.data  # Формат: [число масок, высота, ширина]
num_masks = masks.shape[0]

# Коралловый цвет для каждой маски
colors = [(90, 80, 252) for _ in range(num_masks)]
font_color = (245, 245, 5)

# Создание изображения для отображения масок
mask_overlay = np.zeros_like(img)

labeled_image = img.copy()


# Добавление подписей и контура к маскам
for i in range(num_masks):
    color = colors[i] # Извлекаем цвет
    mask = masks[i].cpu()
    
    # Изменение размера маски до размеров исходного изображения с использованием метода ближайших соседей
    mask_resized = cv2.resize(np.array(mask), (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)
    
    # Получение класса для текущей маски
    class_index = int(classes[i])
    class_name = class_names[class_index]
    
    # Добавление подписи к маске
    mask_contours, _ = cv2.findContours(mask_resized.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    temp = np.array(mask_contours).flatten().tolist()
    # print(f"{temp=}")
    # Накладываем видимость контуров
    cv2.drawContours(labeled_image, mask_contours, -1, color, 1)
    
    x_coord = mask_contours[0][:, 0, 0].mean() # Минимум по X (лево-право)
    y_coord = mask_contours[0][:, 0, 1].mean() # Середина по Y (вверх-вниз)
    cv2.putText(labeled_image, f"{confs[i]:.2f}", (int(x_coord), int(y_coord)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, font_color, 1)

# Отобразите итоговое изображение с наложенными масками и подписями
plt.figure(figsize=(8, 8), dpi=150)
labeled_image = cv2.cvtColor(labeled_image, cv2.COLOR_BGR2RGB)
plt.imshow(labeled_image)
plt.axis('off')
plt.show()

In [ ]:
# Координаты боксов
# print(results[0].boxes.xywhn)
# Форма тензора выходного
# print(results[0].boxes.xywhn.shape)
# Метки
# print(results[0].names)
# Уверенность в боксах
print(results[0].boxes.conf)

# Маска найденного 0 элемента (data[0])
plt.imshow(results[0].masks.data[1].cpu().numpy(), 'gray');